### Question_1


1. Report the calibration matrix for the camera chosen and verify (using an example) the same.

In [2]:
import cv2
import numpy as np

# Load image
img = cv2.imread('img_5.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Find the chess board corners
ret, corners = cv2.findChessboardCorners(gray, (7,6), None)  # Adjust the (7,6) based on your chessboard

# If found, add object points, image points
if ret == True:
    cv2.drawChessboardCorners(img, (7,6), corners, ret)
    cv2.imshow('img', img)
    cv2.waitKey(500)


In [4]:
import numpy as np
import cv2
import glob

# Termination criteria for corner refinement
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

chessboard_size = (7, 6)
square_size = 25  
objp = np.zeros((chessboard_size[0]*chessboard_size[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2) * square_size

# Arrays to store object points and image points from all images.
object_points = []  # 3d points in real world space
image_points = []  # 2d points in image plane.

# Read images
images = glob.glob('E:\GSU\Course Work\Computer Vision\Assignment_1\images\*.jpg')  # Adjust the path to where your images are stored

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    # If found, add object points, image points
    if ret == True:
        object_points.append(objp)

        # Refine the corner positions
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        image_points.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, chessboard_size, corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()


In [5]:
# Assuming object_points and image_points are collected from all images
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, gray.shape[::-1], None, None)


In [6]:
print("Camera matrix:\n", mtx)

# Verification by re-projecting points
example_img = image_points[0]  
example_objp = object_points[0]
_, rvec, tvec = cv2.solvePnP(example_objp, example_img, mtx, dist)
projected_img_points, _ = cv2.projectPoints(example_objp, rvec, tvec, mtx, dist)

# Calculate and print error
error = cv2.norm(example_img, projected_img_points, cv2.NORM_L2) / len(projected_img_points)
print("Reprojection error:", error)


Camera matrix:
 [[766.36635508   0.         341.1906271 ]
 [  0.         764.59455126 246.10584627]
 [  0.           0.           1.        ]]
Reprojection error: 0.09827432309312015


### Question_2


2. Point the camera to a chessboard pattern or any known set of reference points that lie on the
same plane. Capture a series of 10 images by changing the orientation of the camera in each
iteration. Select any 1 image, and using the image formation pipeline equation, set up the linear
equations in matrix form and solve for intrinsic and extrinsic parameters (extrinsic for that
particular orientation). You will need to make measurements of the actual 3D world points, and
mark pixel coordinates. Once you compute the Rotation matrix, you also need to compute the
angles of rotation along each axis. Choose your order of rotation based on your experimentation
setup.

In [15]:
import cv2
import numpy as np

# Load an image
image = cv2.imread('img_5.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Chessboard corners
ret, corners = cv2.findChessboardCorners(gray, (7, 6), None)
if ret:
    corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

objp = np.zeros((7*6, 3), np.float32)
objp[:, :2] = np.mgrid[0:7, 0:6].T.reshape(-1, 2)

# As intrinsic matrix K is calculated previously
f_x = 766.36635508 #the focal length in terms of pixels along the x-axis.
f_y = 764.59455126 #the focal length in terms of pixels along the y-axis.
c_x = 341.1906271 #the x-coordinate of the principal point (optical center).
c_y = 246.10584627 #the y-coordinate of the principal point (optical center).

K = np.array([[f_x, 0, c_x], [0, f_y, c_y], [0, 0, 1]])  

ret, rvec, tvec = cv2.solvePnP(objp, corners2, K, None)

# Convert rotation vector to rotation matrix
R, _ = cv2.Rodrigues(rvec)

# Calculate Euler angles from R
theta_x = np.arctan2(R[2,1], R[2,2])
theta_y = np.arctan2(-R[2,0], np.sqrt(R[2,1]**2 + R[2,2]**2))
theta_z = np.arctan2(R[1,0], R[0,0])

print("Rotation Angles: \n", "theta_x:", theta_x, "\n theta_y:",theta_y, "\n theta_z:", theta_z)


Rotation Angles: 
 theta_x: -0.09284054434550042 
 theta_y: 0.06772383446126788 
 theta_z: 0.18612752910661196


### Question_3


3. Write a script to find the real world dimensions (e.g. diameter of a ball, side length of a cube)
of an object using perspective projection equations. Validate using an experiment where you
image an object using your camera from a specific distance (choose any distance but ensure you
are able to measure it accurately) between the object and camera.

In [16]:
import cv2
import numpy as np

def calculate_real_world_dimension(pixel_width, pixel_height, known_distance, focal_length):
    """
    Calculate the real-world dimensions of an object from its image dimensions.
    
    :param pixel_width: Width of the object in pixels
    :param pixel_height: Height of the object in pixels
    :param known_distance: Distance from camera to object (mm)
    :param focal_length: Focal length of the camera (mm)
    :return: Real world width and height of the object (mm)
    """
    real_width = (pixel_width * known_distance) / focal_length
    real_height = (pixel_height * known_distance) / focal_length
    return real_width, real_height

# Load the image
image = cv2.imread('img_5.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Placeholder values for bounding box coordinates 
x1, y1 = 100, 150
x2, y2 = 300, 350

# Calculate pixel dimensions
pixel_width = abs(x2 - x1)
pixel_height = abs(y2 - y1)

# Known parameters 
known_distance = 1000  # mm
focal_length = 766.36  # mm, using f_x from camera matrix

# Calculate real world dimensions
real_width, real_height = calculate_real_world_dimension(pixel_width, pixel_height, known_distance, focal_length)
print(f"Real World Dimensions: Width = {real_width} mm, Height = {real_height} mm")


Real World Dimensions: Width = 260.973954799311 mm, Height = 260.973954799311 mm


### Question_4

4. Write an application – must run as a Web application on a browser and be OS agnostic – that
implements the solution for problem (3) [An application that can compute real-world dimensions
of an object in view]. Make justifiable assumptions (e.g. points of interest on the object can be
found by clicking on the view or touching on the screen).